In [1]:
import os
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model, ensemble, tree
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score,GridSearchCV,cross_validate
from sklearn.metrics import confusion_matrix, auc,roc_auc_score,roc_curve,recall_score,classification_report
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100, 'display.max_columns', 400)


import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, auc,roc_auc_score,roc_curve,recall_score,classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import joblib
from mlxtend.plotting import plot_confusion_matrix

import warnings
warnings.filterwarnings("ignore")


import theano
import theano.tensor as Tensor
import random
import sys
import csv
import time
import utils
import visualize
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, BatchNormalization, Bidirectional, LayerNormalization
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import regularizers
import re
from keras.layers import Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.layers import Activation, Dense, Flatten 

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
#Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Bidirectional, GRU, RepeatVector, Dense, TimeDistributed,RNN # for creating layers inside the Neural Network
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


Tensorflow/Keras: 2.9.0


In [3]:
data4 = pd.read_csv('data4.csv')

In [4]:
data4 = data4.drop(['Unnamed: 0'], axis = 1)

In [5]:
train,test= train_test_split(data4, test_size=0.3,random_state=123, stratify=data4.Result)# stratify the outcome
X_train =train.drop(['Result'], axis = 1)
X_test = test.drop(['Result'], axis = 1)
Y_train = train['Result'].values
Y_test = test['Result'].values

In [6]:
print("Before OverSampling, counts of label '1': {}".format(sum(Y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(Y_train == 0)))
  
# import SMOTE module from imblearn library
# pip install imblearn (if you don't have imblearn in your system)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X_train_res, Y_train_res = sm.fit_resample(X_train, Y_train.ravel())
  
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(Y_train_res.shape))
  
print("After OverSampling, counts of label '1': {}".format(sum(Y_train_res == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(Y_train_res == 0)))

Before OverSampling, counts of label '1': 202311
Before OverSampling, counts of label '0': 721368 

After OverSampling, the shape of train_X: (1442736, 63)
After OverSampling, the shape of train_y: (1442736,) 

After OverSampling, counts of label '1': 721368
After OverSampling, counts of label '0': 721368


In [7]:
X_train_res= np.expand_dims(X_train_res, axis=2)

In [8]:
X_train_res.shape

(1442736, 63, 1)

In [9]:
X_test = np.expand_dims(X_test, axis=2)

In [10]:
X_test.shape

(395863, 63, 1)

In [11]:
import keras.backend as K
from keras import models 

def f1_score(y_true, y_pred):

    #Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    #If there are no true samples, fix the F1 score at 0.
    if c3 == 0.0:
        return 0.0

    #How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score 

In [15]:
model = keras.Sequential()
# GRU LAYER IS ADDED TO MODEL WITH 128 CELLS IN IT
model.add(GRU(128, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))
model.add(Dropout(0.2))  # 20% DROPOUT ADDED FOR REGULARIZATION
model.add(BatchNormalization())
model.add(GRU(128, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(GRU(64, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=False))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))  # FINAL CLASSIFICATION LAYER WITH 2 CLASSES AND SOFTMAX
# ---------------------------------------------------------------------------------------------------
# OPTIMIZER SETTINGS
opt = tf.keras.optimizers.Adam(learning_rate=0.0017, decay=1e-6)

# MODEL COMPILE
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy','Recall','Precision',f1_score])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_3 (GRU)                 (None, 63, 128)           50304     
                                                                 
 dropout_3 (Dropout)         (None, 63, 128)           0         
                                                                 
 batch_normalization_3 (Batc  (None, 63, 128)          512       
 hNormalization)                                                 
                                                                 
 gru_4 (GRU)                 (None, 63, 128)           99072     
                                                                 
 dropout_4 (Dropout)         (None, 63, 128)           0         
                                                                 
 batch_normalization_4 (Batc  (None, 63, 128)          512       
 hNormalization)                                      

In [16]:
# RUN THE MODEL
history = model.fit(X_train_res, Y_train_res, epochs=10, batch_size=512, verbose=1,
                    validation_data=(X_test, Y_test))

Epoch 1/10
2818/2818 [==============================] - 1460s 516ms/step - loss: 0.5054 - accuracy: 0.7230 - recall: 0.6708 - precision: 0.7490 - f1_score: 0.7079 - val_loss: 0.4691 - val_accuracy: 0.7644 - val_recall: 0.5795 - val_precision: 0.4694 - val_f1_score: 0.5174
Epoch 2/10
2818/2818 [==============================] - 1397s 496ms/step - loss: 0.3367 - accuracy: 0.8349 - recall: 0.8153 - precision: 0.8485 - f1_score: 0.8310 - val_loss: 0.2202 - val_accuracy: 0.9018 - val_recall: 0.8652 - val_precision: 0.7339 - val_f1_score: 0.7935
Epoch 3/10
2818/2818 [==============================] - 1295s 460ms/step - loss: 0.1780 - accuracy: 0.9217 - recall: 0.9219 - precision: 0.9215 - f1_score: 0.9216 - val_loss: 0.1425 - val_accuracy: 0.9355 - val_recall: 0.9206 - val_precision: 0.8106 - val_f1_score: 0.8617
Epoch 4/10
2818/2818 [==============================] - 1290s 458ms/step - loss: 0.1412 - accuracy: 0.9380 - recall: 0.9424 - precision: 0.9342 - f1_score: 0.9382 - val_loss: 0.1314